# CNN classification MNIST

One Convolutional Layer = `nn.Conv2d` + `nn.Relu` + `nn.MaxPool2d`
### Design Idea
2 Convolutional Layer + Fully-connected Layer

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28) # batch size * channel * height * width
print(inputs.shape)

torch.Size([1, 1, 28, 28])


In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
out = conv1(inputs)
print("First conv1: {}".format(out.shape))
out = pool(out)
print("First pool:{}".format(out.shape))
out = conv2(out)
print("Second conv2: {}".format(out.shape))
out = pool(out)
print("Second pool:{}".format(out.shape))

First conv1: torch.Size([1, 32, 28, 28])
First pool:torch.Size([1, 32, 14, 14])
Second conv2: torch.Size([1, 64, 14, 14])
Second pool:torch.Size([1, 64, 7, 7])


In [7]:
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [8]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


You can compute a matrix of vector size of (1, 10)

In [21]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [22]:
torch.manual_seed(42)

In [23]:
lr = 0.001
training_epochs = 15
batch_size = 100

In [24]:
mnist_train = dsets.MNIST(root='MNIST_data/',train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',train=False, transform=transforms.ToTensor(), download=True)

In [25]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True,drop_last=True)

In [26]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
                                    )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [27]:
model = CNN()

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [29]:
total_batch = len(data_loader)
print(total_batch)

600


In [30]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print("Epoch: {}, cost={}".format(epoch+1, avg_cost))

Epoch: 1, cost=0.24108460545539856
Epoch: 2, cost=0.05963026359677315
Epoch: 3, cost=0.0443294420838356
Epoch: 4, cost=0.035916462540626526
Epoch: 5, cost=0.029015548527240753


KeyboardInterrupt: 